In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam
import os

# Define Autoencoder
def build_autoencoder(input_shape):
    input_img = Input(shape=input_shape)

    # Encoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return autoencoder

# Preprocess Video Frames
def preprocess_frame(frame, target_size):
    frame_resized = cv2.resize(frame, target_size)
    return frame_resized / 255.0  # Normalize to [0, 1]

# Postprocess Frames
def postprocess_frame(frame, original_size):
    frame_rescaled = (frame * 255).astype(np.uint8)  # Rescale to [0, 255]
    return cv2.resize(frame_rescaled, original_size)

# Detect Anomalies
def detect_anomalies(original_frame, reconstructed_frame):
    # Compute the difference between original and reconstructed frames
    anomaly_map = np.abs(original_frame - reconstructed_frame)
    anomaly_map = np.mean(anomaly_map, axis=-1)  # Convert to single-channel
    anomaly_map = (anomaly_map * 255).astype(np.uint8)  # Rescale to [0, 255]
    return anomaly_map

# Process Video with Anomalies
def process_video_with_anomalies(input_video_path, output_video_path, model, input_size, batch_size=100, epochs_per_batch=5, model_save_path='autoencoder_model.h5'):
    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    frame_batch = []  # Batch of frames for incremental training

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame
        preprocessed_frame = preprocess_frame(frame, input_size)
        frame_batch.append(preprocessed_frame)

        # Process the frame using the model
        preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)
        reconstructed_frame = model.predict(preprocessed_frame)
        reconstructed_frame = reconstructed_frame[0]

        # Detect anomalies
        anomalies = detect_anomalies(preprocessed_frame[0], reconstructed_frame)
        anomaly_overlay = cv2.applyColorMap(anomalies, cv2.COLORMAP_JET)

        # Resize anomaly overlay to match the original frame dimensions
        anomaly_overlay_resized = cv2.resize(anomaly_overlay, (frame.shape[1], frame.shape[0]))

        # Combine original frame and resized anomaly overlay
        combined_frame = cv2.addWeighted(frame, 0.7, anomaly_overlay_resized, 0.3, 0)

        # Write frame to output video
        out.write(combined_frame)

        # Perform incremental training if batch is full
        if len(frame_batch) == batch_size:
            print(f"Training on new batch of {batch_size} frames...")
            incremental_train(model, frame_batch, epochs=epochs_per_batch)
            frame_batch = []  # Clear the batch

            # Save the model after incremental training
            model.save(model_save_path)
            print(f"Model saved to {model_save_path}")

    # Final cleanup
    cap.release()
    out.release()

# Incremental Training
def incremental_train(model, frames, epochs=1):
    frames = np.array(frames)
    model.fit(frames, frames, epochs=epochs, verbose=1)

# Main Function
if __name__ == "__main__":
    input_video_path = '/content/4711774-uhd_3840_2160_30fps.mp4'  # Path to input video
    output_video_path = 'output.mp4'  # Path to save output video
    input_size = (128, 128, 3)  # Resize frames to this size
    model_save_path = 'autoencoder_model.h5'  # Path to save the trained model

    # Initialize Autoencoder
    autoencoder = None
    if os.path.exists(model_save_path):
        print(f"Loading existing model from {model_save_path}...")
        try:
            autoencoder = tf.keras.models.load_model(
                model_save_path,
                custom_objects={"MeanSquaredError": tf.keras.losses.MeanSquaredError}
            )
            print("Model loaded successfully!")
        except Exception as e:
            print(f"Error loading model: {e}. Rebuilding model.")
            autoencoder = build_autoencoder(input_size)
    else:
        print("No existing model found. Starting from scratch.")
        autoencoder = build_autoencoder(input_size)

    # Process video with anomaly detection
    process_video_with_anomalies(input_video_path, output_video_path, autoencoder, input_size[:2], batch_size=100, epochs_per_batch=5, model_save_path=model_save_path)

No existing model found. Starting from scratch.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1

Model saved to autoencoder_model.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━

Model saved to autoencoder_model.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━

Model saved to autoencoder_model.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━

Model saved to autoencoder_model.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━

In [2]:
import cv2
import numpy as np
import tensorflow as tf
import os

# Preprocess Video Frames
def preprocess_frame(frame, target_size):
    frame_resized = cv2.resize(frame, target_size)
    return frame_resized / 255.0  # Normalize to [0, 1]

# Postprocess Frames
def postprocess_frame(frame, original_size):
    frame_rescaled = np.clip(frame * 255, 0, 255).astype(np.uint8)  # Rescale to [0, 255]
    return cv2.resize(frame_rescaled, original_size)

# Detect Anomalies
def detect_anomalies(original_frame, reconstructed_frame, anomaly_threshold=100):
    # Compute the difference between original and reconstructed frames
    anomaly_map = np.abs(original_frame - reconstructed_frame)
    anomaly_map = np.mean(anomaly_map, axis=-1)  # Convert to single-channel
    anomaly_map = (anomaly_map * 255).astype(np.uint8)  # Rescale to [0, 255]

    # Threshold the anomaly map to identify significant anomalies
    _, thresh = cv2.threshold(anomaly_map, anomaly_threshold, 255, cv2.THRESH_BINARY)

    # Find contours of anomalies
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return anomaly_map, contours

# Draw Bounding Boxes Around Anomalous Regions
def draw_bounding_boxes(frame, contours):
    # Draw bounding boxes for detected anomalies
    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Ignore small contours (noise)
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green bounding box
    return frame

# Process Video with Anomalies (Prediction Only)
def predict_anomalies_in_video(input_video_path, output_video_path, model, input_size):
    cap = cv2.VideoCapture(input_video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame
        preprocessed_frame = preprocess_frame(frame, input_size)
        preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)

        # Predict reconstructed frame
        reconstructed_frame = model.predict(preprocessed_frame)
        reconstructed_frame = postprocess_frame(reconstructed_frame[0], (width, height))

        # Detect anomalies and find contours
        anomalies, contours = detect_anomalies(frame / 255.0, reconstructed_frame / 255.0)

        # Draw bounding boxes on the original frame around anomalous regions
        frame_with_boxes = draw_bounding_boxes(frame.copy(), contours)

        # Resize anomaly map to match the original frame dimensions
        anomaly_overlay = cv2.applyColorMap(anomalies, cv2.COLORMAP_JET)
        anomaly_overlay_resized = cv2.resize(anomaly_overlay, (frame.shape[1], frame.shape[0]))

        # Combine original frame and resized anomaly overlay
        combined_frame = cv2.addWeighted(frame_with_boxes, 0.7, anomaly_overlay_resized, 0.3, 0)

        # Write frame to output video
        out.write(combined_frame)

    # Final cleanup
    cap.release()
    out.release()

# Main Function
if __name__ == "__main__":
    input_video_path = '/content/4711774-uhd_3840_2160_30fps.mp4'  # Path to input video
    output_video_path = 'anot.mp4'  # Path to save output video
    model_save_path = 'autoencoder_model.h5'  # Path to saved pre-trained model
    input_size = (128, 128, 3)  # Size the model expects

    # Load the pre-trained model
    if not os.path.exists(model_save_path):
        print(f"Model not found at {model_save_path}. Please train the model first.")
        exit()

    print(f"Loading model from {model_save_path}...")
    autoencoder = tf.keras.models.load_model(model_save_path, compile=False)
    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')

    # Predict anomalies in the video
    predict_anomalies_in_video(input_video_path, output_video_path, autoencoder, input_size[:2])

Loading model from autoencoder_model.h5...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━